# IBM Applied Data Science Capstone Course by Coursera
### Week 5 Final Report
**_Opening a New Shopping Mall in Bogotá, Colombia_**
- Build a dataframe of neighborhoods in Bogotá, Colombia by web scraping the data from Wikipedia page:
https://es.wikipedia.org/wiki/Unidades_de_Planeamiento_Zonal
- Get the geographical coordinates of the neighborhoods
- Obtain the venue data for the neighborhoods from Foursquare API
- Explore and cluster the neighborhoods
- Select the best cluster to open a new shopping mall
***
### 1. Import libraries

In [28]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents
import urllib # package required to interact with live webpage

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


### 2. Scrap data from Wikipedia page into a DataFrame

In [200]:
# Scrape data from website
response = requests.get('https://es.wikipedia.org/wiki/Unidades_de_Planeamiento_Zonal')
soup = BeautifulSoup(response.text)

borough_data = {}
table_body = soup.find('table', {'class': ['wikitable']}).find('tbody')

for row in table_body.find_all('tr'):
    cells = row.find_all('td')
    if cells:
        neighborhood = cells[1].text.strip()
        number = int(cells[0].text.strip())
        borough = cells[2].text.strip()
        borough_data[number] = {
           'Neighborhood': neighborhood,
           'Borough': borough
            }

In [201]:
# create a new DataFrame from the list
kl_df = pd.DataFrame.from_dict(borough_data, orient='index')
kl_df.head()

,Neighborhood,Borough
1,Paseo de los Libertadores,01 Usaquén
9,Verbenal,01 Usaquén
10,La Uribe,01 Usaquén
11,San Cristóbal Norte,01 Usaquén
12,Toberín,01 Usaquén


In [202]:
# print the number of rows of the dataframe
kl_df.shape

(112, 2)

### 3. Get the geographical coordinates

In [215]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Bogotá, Colombia'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [216]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist()]

In [249]:
coords[:10]

[[4.779711913265339, -74.04720298432989],
 [4.72735004832921, -74.06482994503435],
 [4.75180000000006, -74.04314999999997],
 [4.739530000000059, -74.02238999999997],
 [4.743150000000071, -74.04197999999997],
 [4.7252900000000295, -74.04783999999995],
 [4.692590000000052, -74.03008999999997],
 [4.712740000000053, -74.04727999999994],
 [4.724340000000041, -74.11721999999997],
 [4.691510000000051, -74.14860999999996]]

In [218]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [219]:
# merge the coordinates into the original dataframe
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']

In [222]:
# check the neighborhoods and the coordinates
print(kl_df.shape)
kl_df = kl_df.dropna()

(112, 4)


In [223]:
# save the DataFrame as CSV file
kl_df.to_csv("kl_df.csv", index=False)

### 4. Create a map of Bogotá with neighborhoods superimposed on top

In [211]:
# get the coordinates of Bogotá
address = 'Bogotá, Colombia'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bogotá, Colombia {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bogotá, Colombia 4.59808, -74.0760439.


In [224]:
# create map of Bogotá using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [225]:
# save the map as HTML file
map_kl.save('map_kl.html')

### 5. Use the Foursquare API to explore the neighborhoods

In [250]:
# define Foursquare Credentials and Version
CLIENT_ID = 'your Foursquare ID' # your Foursquare ID
CLIENT_SECRET = 'your Foursquare Secret' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: your Foursquare ID
CLIENT_SECRET:your Foursquare Secret


**Now, let's get the top 100 venues that are within a radius of 2000 meters.**

In [227]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [228]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(6204, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Paseo de los Libertadores,4.72735,-74.06483,Sagal La Colina,4.726559,-74.061541,Steakhouse
1,Paseo de los Libertadores,4.72735,-74.06483,Buffalo Wings,4.723067,-74.062448,Wings Joint
2,Paseo de los Libertadores,4.72735,-74.06483,WRIBS,4.726935,-74.064903,Sports Bar
3,Paseo de los Libertadores,4.72735,-74.06483,Juan Valdez Café,4.728148,-74.064754,Café
4,Paseo de los Libertadores,4.72735,-74.06483,Cinemark,4.731746,-74.066599,Movie Theater


**Let's check how many venues were returned for each neighorhood**

In [229]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
12 de Octubre,100,100,100,100,100,100
20 de Julio,100,100,100,100,100,100
Alfonso López,70,70,70,70,70,70
Américas,12,12,12,12,12,12
Apogeo,81,81,81,81,81,81
Arborizadora,100,100,100,100,100,100
Bolivia,100,100,100,100,100,100
Bosa Central,100,100,100,100,100,100
Bosa Occidental,100,100,100,100,100,100


**Let's find out how many unique categories can be curated from all the returned venues**

In [230]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 245 uniques categories.


In [231]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Steakhouse', 'Wings Joint', 'Sports Bar', 'Café', 'Movie Theater',
       'Clothing Store', 'Pharmacy', 'Vegetarian / Vegan Restaurant',
       'Paper / Office Supplies Store', 'Sporting Goods Shop',
       'Asian Restaurant', 'Pizza Place', 'Department Store',
       'Burger Joint', 'Shopping Mall', 'Pub', 'Fast Food Restaurant',
       'Pastry Shop', 'Bar', 'Coffee Shop', 'Furniture / Home Store',
       'Nightlife Spot', 'Creperie', 'Japanese Restaurant', 'Food Court',
       'Spanish Restaurant', 'Supermarket', 'Salad Place', 'Lounge',
       'BBQ Joint', 'Brewery', 'General Entertainment',
       'Italian Restaurant', 'Bakery', 'Farm', 'Donut Shop', 'Gym',
       'Gift Shop', 'Bubble Tea Shop', 'American Restaurant',
       'Bike Trail', 'Pet Store', 'Park', 'Empanada Restaurant',
       'Restaurant', 'Deli / Bodega', 'Soccer Field', 'Dance Studio',
       'Gymnastics Gym', 'Seafood Restaurant'], dtype=object)

In [232]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

### 6. Analyze Each Neighborhood

In [233]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(6204, 246)


,Neighborhoods,Accessories Store,Advertising Agency,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bike Trail,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Bowling Green,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Butcher,Café,Cajun / Creole Restaurant,Campground,Candy Store,Car Wash,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Circus,Clothing Store,Cocktail Bar,Coffee Shop,College Bookstore,College Soccer Field,Colombian Restaurant,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Doctor's Office,Dog Run,Donut Shop,Drugstore,Duty-free Shop,Electronics Store,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Event Space,Exhibit,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Fishing Store,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Field,Home Service,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Kids Store,Latin American Restaurant,Library,Lounge,Luggage Store,Market,Martial Arts Dojo,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nature Preserve,Neighborhood,Nightclub,Nightlife Spot,Optical Shop,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Outlet Mall,Paintball Field,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pie Shop,Pizza Place,Planetarium,Playground,Plaza,Pool,Pub,Public Art,Racetrack,Record Shop,Recreation Center,Rental Car Location,Rental Service,Resort,Restaurant,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Stadium,Steakhouse,Student Center,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Toy / Game Store,Trade School,Trail,Train Station,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Veterinarian,Warehouse Store,Water Park,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Paseo de los Libertadores,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Paseo de los Libertadores,0,0,0,0,0,0,0,0,0,

**Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [234]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(106, 246)


,Neighborhoods,Accessories Store,Advertising Agency,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bike Trail,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Bowling Green,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Butcher,Café,Cajun / Creole Restaurant,Campground,Candy Store,Car Wash,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Circus,Clothing Store,Cocktail Bar,Coffee Shop,College Bookstore,College Soccer Field,Colombian Restaurant,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Doctor's Office,Dog Run,Donut Shop,Drugstore,Duty-free Shop,Electronics Store,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Event Space,Exhibit,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Fishing Store,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Field,Home Service,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Kids Store,Latin American Restaurant,Library,Lounge,Luggage Store,Market,Martial Arts Dojo,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nature Preserve,Neighborhood,Nightclub,Nightlife Spot,Optical Shop,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Outlet Mall,Paintball Field,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pie Shop,Pizza Place,Planetarium,Playground,Plaza,Pool,Pub,Public Art,Racetrack,Record Shop,Recreation Center,Rental Car Location,Rental Service,Resort,Restaurant,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Stadium,Steakhouse,Student Center,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Toy / Game Store,Trade School,Trail,Train Station,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Veterinarian,Warehouse Store,Water Park,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,12 de Octubre,0.00,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.00,0.000000,0.020000,0.01,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.010000,0.020000,0.000000,0.010000,0.02,0.000000,0.000000,0.000000,0.00,0.01,0.020000,0.00,0.00,0.010000,0.00,0.00,0.020000,0.020000,0.000000,0.00,0.000000,0.010000,0.000000,0.000000,0.00,0.050000,0.000000,0.000000,0.00,0.000000,0.010000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.050000,0.00,0.000000,0.00,0.020000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0

In [235]:
len(kl_grouped[kl_grouped["Shopping Mall"] > 0])

61

**Create a new DataFrame for Shopping Mall data only**

In [236]:
kl_mall = kl_grouped[["Neighborhoods","Shopping Mall"]]

In [237]:
kl_mall.head()

,Neighborhoods,Shopping Mall
0,12 de Octubre,0.000000
1,20 de Julio,0.020000
2,Alfonso López,0.042857
3,Américas,0.000000
4,Apogeo,0.024691


### 7. Cluster Neighborhoods
Run k-means to cluster the neighborhoods in Bogotá into 3 clusters.

In [239]:
# set number of clusters
kclusters = 3

kl_clustering = kl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 2, 0, 2, 2, 0, 2, 0, 0], dtype=int32)

In [240]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [241]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,12 de Octubre,0.000000,0
1,20 de Julio,0.020000,2
2,Alfonso López,0.042857,2
3,Américas,0.000000,0
4,Apogeo,0.024691,2


In [242]:
# merge grouped with data to add latitude/longitude for each neighborhood
kl_merged = kl_merged.join(kl_df.set_index("Neighborhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(106, 6)


,Neighborhood,Shopping Mall,Cluster Labels,Borough,Latitude,Longitude
0,12 de Octubre,0.000000,0,12 Barrios Unidos,4.64084,-74.06872
1,20 de Julio,0.020000,2,04 San Cristóbal,4.63320,-74.09438
2,Alfonso López,0.042857,2,05 Usme,4.70282,-74.09522
3,Américas,0.000000,0,08 Kennedy,4.64335,-74.16707
4,Apogeo,0.024691,2,07 Bosa,4.63814,-74.13753


In [243]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(106, 6)


,Neighborhood,Shopping Mall,Cluster Labels,Borough,Latitude,Longitude
0,12 de Octubre,0.000000,0,12 Barrios Unidos,4.640840,-74.068720
38,Gran Britalia,0.010000,0,08 Kennedy,4.663290,-74.070340
39,Gran Yomasa,0.000000,0,05 Usme,4.697930,-74.141140
42,Jardín Botánico,0.000000,0,10 Engativá,4.505270,-74.117200
44,Kennedy Central,0.000000,0,08 Kennedy,4.639300,-74.161200
48,La Esmeralda,0.000000,0,13 Teusaquillo,4.567840,-74.161720
104,Zona Franca,0.000000,0,09 Fontibón,4.728370,-74.088350
53,La Sabana,0.000000,0,14 Mártires,4.553540,-74.114370
54,La Uribe,0.000000,0,01 Usaquén,4.666400,-74.018970
55,Las Cruces,0.000000,0,03 Santa Fe,4.548420,-74.149550


**Finally, let's visualize the resulting clusters**

In [244]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [245]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### 8. Examine Clusters

#### Cluster 0

In [246]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Borough,Latitude,Longitude
0,12 de Octubre,0.00,0,12 Barrios Unidos,4.640840,-74.068720
38,Gran Britalia,0.01,0,08 Kennedy,4.663290,-74.070340
39,Gran Yomasa,0.00,0,05 Usme,4.697930,-74.141140
42,Jardín Botánico,0.00,0,10 Engativá,4.505270,-74.117200
44,Kennedy Central,0.00,0,08 Kennedy,4.639300,-74.161200
48,La Esmeralda,0.00,0,13 Teusaquillo,4.567840,-74.161720
104,Zona Franca,0.00,0,09 Fontibón,4.728370,-74.088350
53,La Sabana,0.00,0,14 Mártires,4.553540,-74.114370
54,La Uribe,0.00,0,01 Usaquén,4.666400,-74.018970
55,Las Cruces,0.00,0,03 Santa Fe,4.548420,-74.149550


#### Cluster 1

In [247]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Borough,Latitude,Longitude
82,San Blas,0.083333,1,04 San Cristóbal,4.56182,-74.12734
17,Ciudad Jardín,0.125000,1,15 Antonio Nariño,4.60931,-74.18482
64,Lucero,0.160000,1,19 Ciudad Bolívar,4.67501,-74.15164
29,El Prado,0.100000,1,11 Suba,4.56788,-74.08392
88,San Rafael,0.071429,1,16 Puente Aranda,4.65249,-74.15913
49,La Flora,0.125000,1,05 Usme,4.67250,-74.15961
86,San José,0.111111,1,18 Rafael Uribe,4.64350,-74.18751
14,Castilla,0.105263,1,08 Kennedy,4.61837,-74.17512
96,Timiza,0.083333,1,08 Kennedy,4.63045,-74.19493
50,La Floresta,0.142857,1,11 Suba,4.53916,-74.11604


#### Cluster 2

In [248]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Borough,Latitude,Longitude
103,Verbenal,0.044118,2,01 Usaquén,4.69151,-74.14861
1,20 de Julio,0.020000,2,04 San Cristóbal,4.63320,-74.09438
80,Restrepo,0.016949,2,15 Antonio Nariño,4.63948,-74.14337
81,Sagrado Corazón,0.016949,2,03 Santa Fe,4.58340,-74.09961
94,Teusaquillo,0.058824,2,13 Teusaquillo,4.56939,-74.11980
7,Bosa Central,0.020000,2,07 Bosa,4.66136,-74.09647
97,Tintal Norte,0.031250,2,08 Kennedy,4.73617,-74.11039
84,San Francisco,0.055556,2,19 Ciudad Bolívar,4.70476,-74.11855
100,Tunjuelito,0.023810,2,06 Tunjuelito,4.70803,-74.12722
5,Arborizadora,0.020000,2,19 Ciudad Bolívar,4.66297,-74.10193


#### Observations:
Most of the shopping malls are concentrated in the central area of Bogotá, with the highest number in cluster 2 and moderate number in cluster 0. On the other hand, cluster 1 has very low number to totally no shopping mall in the neighborhoods. This represents a great opportunity and high potential areas to open new shopping malls as there is very little to no competition from existing malls. Meanwhile, shopping malls in cluster 2 are likely suffering from intense competition due to oversupply and high concentration of shopping malls. From another perspective, this also shows that the oversupply of shopping malls mostly happened in the central area of the city, with the suburb area still have very few shopping malls. Therefore, this project recommends property developers to capitalize on these findings to open new shopping malls in neighborhoods in cluster 1 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new shopping malls in neighborhoods in cluster 0 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 2 which already have high concentration of shopping malls and suffering from intense competition.